In [1]:
%run common.ipynb

[tokenizer.decode(i) for i in tokenizer.get_batch_data(prefix=True)[1]][:10]

['S字母:3393=qetueetueetueetuwE',
 'S大写:3286=壹叁壹肆伍叁壹肆伍叁壹肆伍叁壹肆肆E',
 'S小写:8046=三二一八七二一八七二一八七二一八四E',
 'S数字:1599=6396639663966396EP',
 'S大写:3849=壹伍叁玖柒伍叁玖柒伍叁玖柒伍叁玖陆E',
 'S字母:1417=tyyityyityyityyiEP',
 'S小写:2652=一〇六〇九〇六〇九〇六〇九〇六〇八E',
 'S数字:2587=10349034903490348E',
 'S大写:1158=肆陆叁贰肆陆叁贰肆陆叁贰肆陆叁贰EP',
 'S字母:1661=yyrryyrryyrryyrrEP']

In [2]:
model_ppo = torch.load('ppo.model')
model_ppo.to(device)
model_ppo.eval()

ModelPPO(
  (model_gen): ModelGEN(
    (feature): GPT2Model(
      (wte): Embedding(53, 64)
      (wpe): Embedding(128, 64)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPT2Block(
          (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (fc_out): Linear(in_features=64, out_features=53, bias=False)
  )
  (v_head): Sequential(
    (0): Dropout(p=0.1, inplace=False)

In [3]:
#随机一批数据
_, input_ids, _ = tokenizer.get_batch_data(prefix=True)

#切分成question和answer
split = [i.index(tokenizer.encoder['=']) + 1 for i in input_ids]
question = [input_ids[i][:split[i]] for i in range(len(input_ids))]
answer = [input_ids[i][split[i]:] for i in range(len(input_ids))]

#根据question生成predict
input_ids = [torch.LongTensor(i).unsqueeze(0).to(device) for i in question]
predict = [generate(model_ppo.model_gen, i) for i in input_ids]

#裁剪,只要生成的部分
predict = [p[0].tolist()[len(q):] for p, q in zip(predict, question)]

#解码成文本
question = [tokenizer.decode(i) for i in question]
answer = [tokenizer.decode(i) for i in answer]
predict = [tokenizer.decode(i) for i in predict]

acc = 0
for q, a, p in zip(question, answer, predict):
    print(q, a, p, a == p)
    if a == p:
        acc += 1

acc / len(question)

S字母:6263= wtptrtptrtptrtptwE wtptrtptrtptrtptwE True
S大写:8733= 叁肆玖叁伍肆玖叁伍肆玖叁伍肆玖叁贰E 叁肆玖叁伍肆玖叁伍肆玖叁伍肆玖叁贰E True
S数字:9847= 39391939193919388E 39391939193919388E True
S小写:7170= 二八六八二八六八二八六八二八六八〇E 二八六八二八六八二八六八二八六八〇E True
S数字:4609= 18437843784378436E 18437843784378436E True
S大写:4292= 壹柒壹陆玖柒壹陆玖柒壹陆玖柒壹陆捌E 壹柒壹陆玖柒壹陆玖柒壹陆玖柒壹陆捌E True
S大写:9570= 叁捌贰捌叁捌贰捌叁捌贰捌叁捌贰捌零E 叁捌贰捌叁捌贰捌叁捌贰捌叁捌贰捌零E True
S小写:6758= 二七〇三四七〇三四七〇三四七〇三二E 二七〇三四七〇三四七〇三四七〇三二E True
S数字:3001= 12005200520052004E 12005200520052004E True
S大写:9290= 叁柒壹陆叁柒壹陆叁柒壹陆叁柒壹陆零E 叁柒壹陆叁柒壹陆叁柒壹陆叁柒壹陆零E True
S大写:9691= 叁捌柒陆柒捌柒陆柒捌柒陆柒捌柒陆肆E 叁捌柒陆柒捌柒陆柒捌柒陆柒捌柒陆肆E True
S数字:5483= 21934193419341932E 21934193419341932E True
S小写:4630= 一八五二一八五二一八五二一八五二〇E 一八五二一八五二一八五二一八五二〇E True
S小写:7920= 三一六八三一六八三一六八三一六八〇E 三一六八三一六八三一六八三一六八〇E True
S数字:5449= 21798179817981796E 21798179817981796E True
S数字:4995= 19981998199819980E 19981998199819980E True
S小写:5020= 二〇〇八二〇〇八二〇〇八二〇〇八〇E 二〇〇八二〇〇八二〇〇八二〇〇八〇E True
S数字:4635= 18541854185418540E 18541854185418540E True
S字母:4359= qureuureuureuureyE qureuureuureuurey

0.921875